# CSE 168 Lab 3 - Option 1

This notebook describes CSE 168 Lab 3 - Option 1 for students Shawn Duong, Chloe Engel, Charison Gill-Branion, and Isabella Montoya in the Fall semester of 2022.

For this lab, we are training a model to detect 4 hand gestures. This follows the tutorial given in the lab handout by Nicholas Renotte.

Before running this notebook, one should set up the venv and install the dependencies as per the tutorial:

```
python -m venv tfod

source tfod/bin/activate # Linux
.\tfod\Scripts\activate # Windows 

python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --user --name=tfodj
```

Make sure that the notebook's kernel is tfodj as well.

# Part 1: Collecting Training Images

## Step 1: Install and Import Dependencies

We must install and import the dependencies. We need opencv-python in order to use computer vision related functionalities.

In [1]:
!pip install opencv-python

In [2]:
import cv2
import os
import time

## Step 2: Define the Images to Collect

We are collecting the hand gestures that make up "Hello World," and saving 5 images per gesture. We can collect more training images by just re-running the code in step 4, though.

In [3]:
# The gestures we are training the model to detect.
labels = ["h", "e", "l", "o", "w", "r", "d"]

# The number of training images per gesture we will take.
nImgs = 5

## Step 3: Set Up File Structure

We are going to save everything in `./tensorflow/workspaces/images/training_images/`.

In [4]:
# The path to store our training images in.
path = "./tensorflow/workspace/images/training_images/"

# Create the path if it does not exist.
if not os.path.exists(path):
    os.makedirs(path)

## Step 4: Capture Training Images From Webcam

We capture 5 images per gesture from the webcam. We can press 'q' on our keyboard to quit early, or 'c' to capture an image when we're ready.

In [5]:
# If we have already trained, we may want to skip this step.
# If you have not trained yet, set this to False.
skipTraining = True

if not skipTraining:
    
    cap = cv2.VideoCapture(0)

    # Loop for all gestures we want to train.
    for label in labels:

        print(f"Capturing images for: {label}")

        completed = 0
        earlyExit = False

        # Loop for however many images we wish to capture per gesture.
        while completed < nImgs:

            # Read from the camera and show it to us.
            _, frame = cap.read()

            try:
                cv2.imshow("Frame", frame)
            except:
                continue

            # Webcam refresh rate.
            time.sleep(0.01)

            # Press 'q' to quit.
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                earlyExit = True
                break

            # Press 'c' to capture.
            elif key == ord('c'):
                # Capture and save the image.
                completed += 1
                print(f"Capturing image {completed}/{nImgs}")
                cv2.imwrite(path+f"{label}_{int(time.time())}.jpg", frame)

        if earlyExit:
            break

    cap.release()
    cv2.destroyAllWindows()

# Part 2: Training and Detection

## Step 1: Download and Compile TFOD

We must download and install TFOD from TensorFlow's GitHub. We must compile all the proto files to do so. This is different for Linux and Windows.

In [6]:
# Make the repository where we will clone the TensorFlow models repo.
if not os.path.exists("./models/"):
    os.makedirs("./models/")
    !git clone https://github.com/tensorflow/models ./models/
    
# For Linux.
if os.name == "posix":
    
    # For Arch Linux.
    if "arch" in os.uname().release:
        !pacman -Syu protobuf
        
    # If you're not using Arch, you're probably on Ubuntu or some
    # other Debian derivative and use apt.
    else:
        !apt-get install protobuf-compiler
        
    # Compile the proto files.
    !cd ./models/research && protoc object_detection/protos/*.proto --python_out=. \
     && cp object_detection/packages/tf2/setup.py . && python -m pip install .

# For Windows.
else:
    
    # TODO.
    pass

error: you cannot perform this operation unless you are root.
Processing /home/skat/doc/repos/cse168-final/models/research
  Preparing metadata (setup.py) ... done


  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1655141 sha256=f62ba00ea5940a282b6b3a0d1c1232c18618289276cc7dbcc861359f16d92c6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ue76p4dn/wheels/82/11/de/d6da7bf64cc9d1f460952b8e1a18880a1d88a6f2fac08bd2aa
Successfully built object-detection
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


## Step 2: Install TensorFlow and Upgrade Protobuf

We will be using TensorFlow, so we should make sure it is installed before proceeding. We should also upgrade protobuf, since older versions may lead to an error about `builder.py`.

In [7]:
# You probably already have this installed, but just in case.
!pip install tensorflow
!pip install protobuf==3.20

  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.57.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-api-core 2.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is inco

## Step 3: Get The Pretrained Model

A pretrained model is available at http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz. We just need to get it and extract it now before we can use it with our object detection.

In [8]:
url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

# Make the directory.
if not os.path.exists("./tensorflow/workspace/pretrained_models/"):
    os.makedirs("./tensorflow/workspace/pretrained_models/")

# For Linux.
if os.name == "posix":
    !wget {url}
    !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz ./tensorflow/workspace/pretrained_models/
    !cd ./tensorflow/workspace/pretrained_models/ && tar xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
    
# For Windows.
else:
    # TODO.
    pass

--2022-12-13 02:38:32--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2607:f8b0:4007:815::2010, 142.250.72.176
Connecting to download.tensorflow.org (download.tensorflow.org)|2607:f8b0:4007:815::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  10.6MB/s    in 1.8s    

2022-12-13 02:38:34 (10.6 MB/s) - 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz' saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320

## Step 4: Create the Label Map

We create a file `./tensorflow/workspace/annotations/label_map.pbtxt` containing our labels.

In [9]:
labels = [
    {"name": "h", "id": 1},
    {"name": "e", "id": 2},
    {"name": "l", "id": 3},
    {"name": "o", "id": 4},
    {"name": "w", "id": 5},
    {"name": "r", "id": 6},
    {"name": "d", "id": 7},
]

data = ""

for label in labels:
    
    name = label["name"]
    idno = label["id"]
    
    data += "item {\n"
    data += f"\tname: '{name}'\n"
    data += f"\tid: {idno}\n"
    data += "}\n"

if not os.path.exists("./tensorflow/workspace/annotations/"):
    os.makedirs("./tensorflow/workspace/annotations/")
    
with open("./tensorflow/workspace/annotations/label_map.pbtxt", "w") as f:
    f.write(data)

## Step 5: Copy the Model Config to the Training Folder

We need to create a training folder and copy the model config over to it before we begin training.

In [10]:
if not os.path.exists("./tensorflow/workspace/models/model/"):
    os.makedirs("./tensorflow/workspace/models/model/")

# Linux.
if os.name == "posix":
    !cp ./tensorflow/workspace/pretrained_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
     ./tensorflow/workspace/models/model/
    
# Windows.
else:
    # TODO.
    pass

## Step 6: Create TF Records

We clone and run the author's scripts to generate the TF records used in the next step.

In [11]:
if not os.path.exists("GenerateTFRecord"):
    !git clone https://github.com/nicknochnack/GenerateTFRecord

!python GenerateTFRecord/generate_tfrecord.py -x "./tensorflow/workspace/images/train" \
 -l "./tensorflow/workspace/annotations/label_map.pbtxt" -o "./tensorflow/workspace/annotations/train.record"
!python GenerateTFRecord/generate_tfrecord.py -x "./tensorflow/workspace/images/test" \
 -l "./tensorflow/workspace/annotations/label_map.pbtxt" -o "./tensorflow/workspace/annotations/test.record"

2022-12-13 02:38:35.670108: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/skat/doc/repos/cse168-final/tfod/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/skat/doc/repos/cse168-final/tfod/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/skat/doc/repos/cse168-final/tfod/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/skat/doc/repos/cse168-final/tfod/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, fro

## Step 7: Import Everything and Update the Config for Transfer Learning

We should import everything needed for training and detection now. We can update the config for transfer learning with our training images. If you get warnings about CPU optimization, ignore them -- it has to do with your hardware.

In [12]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2022-12-13 02:38:40.396455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 02:38:40.538846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/skat/doc/repos/cse168-final/tfod/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-13 02:38:40.538873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-13 02:38:40.569198: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [13]:
config = config_util.get_configs_from_pipeline_file("./tensorflow/workspace/models/model/pipeline.config")
pconfig = pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile("./tensorflow/workspace/models/model/pipeline.config", "r") as f:
    pstr = f.read()
    text_format.Merge(pstr, pconfig)
    
pconfig.model.ssd.num_classes = len(labels)
pconfig.train_config.batch_size = 4
pconfig.train_config.fine_tune_checkpoint = "./tensorflow/workspace/pretrained_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
pconfig.train_config.fine_tune_checkpoint_type = "detection"
pconfig.train_input_reader.label_map_path= "./tensorflow/workspace/annotations/label_map.pbtxt"
pconfig.train_input_reader.tf_record_input_reader.input_path[:] = ["./tensorflow/workspace/annotations/train.record"]
pconfig.eval_input_reader[0].label_map_path = "./tensorflow/workspace/annotations/label_map.pbtxt"
pconfig.eval_input_reader[0].tf_record_input_reader.input_path[:] = ["./tensorflow/workspace/annotations/test.record"]